<a href="https://colab.research.google.com/github/mdzikrim/DeepLearning/blob/main/MLP_Model_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MLP PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # flatten 3x32x32 = 3072
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:03<00:00, 45.2MB/s]


In [3]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3072, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 256), nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP().to(device)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(30):  # cukup banyak epoch karena MLP biasanya lambat belajar
    model.train()
    total_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 1468.9574
Epoch 2, Loss: 1327.7686
Epoch 3, Loss: 1254.6986
Epoch 4, Loss: 1203.5891
Epoch 5, Loss: 1160.4313
Epoch 6, Loss: 1129.3663
Epoch 7, Loss: 1108.9030
Epoch 8, Loss: 1083.6387
Epoch 9, Loss: 1059.9580
Epoch 10, Loss: 1037.8052
Epoch 11, Loss: 1016.5132
Epoch 12, Loss: 993.5085
Epoch 13, Loss: 979.0349
Epoch 14, Loss: 957.1929
Epoch 15, Loss: 933.6479
Epoch 16, Loss: 918.1438
Epoch 17, Loss: 899.1369
Epoch 18, Loss: 881.3903
Epoch 19, Loss: 864.5558
Epoch 20, Loss: 848.6776
Epoch 21, Loss: 828.9011
Epoch 22, Loss: 814.2389
Epoch 23, Loss: 798.3927
Epoch 24, Loss: 782.4071
Epoch 25, Loss: 761.8531
Epoch 26, Loss: 747.5804
Epoch 27, Loss: 732.8574
Epoch 28, Loss: 718.7307
Epoch 29, Loss: 698.1826
Epoch 30, Loss: 688.9353


In [5]:
model.eval()
all_preds, all_probs, all_labels = [], [], []

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        preds = np.argmax(probs, axis=1)

        all_probs.extend(probs)
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

accuracy  = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')
recall    = recall_score(all_labels, all_preds, average='macro')
f1        = f1_score(all_labels, all_preds, average='macro')
auc       = roc_auc_score(np.eye(10)[all_labels], all_probs, average='macro', multi_class='ovr')

print("📊 Evaluasi MLP:")
print(f"Akurasi  : {accuracy:.4f}")
print(f"Presisi  : {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")
print(f"AUC      : {auc:.4f}")


📊 Evaluasi MLP:
Akurasi  : 0.4963
Presisi  : 0.5060
Recall   : 0.4963
F1-Score : 0.4916
AUC      : 0.8751


#MLP Tensorflow

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalisasi & Flatten gambar 32x32x3 → 3072
x_train = x_train.reshape(-1, 32*32*3) / 255.0
x_test  = x_test.reshape(-1, 32*32*3) / 255.0

# Ubah label jadi one-hot
y_train_onehot = tf.keras.utils.to_categorical(y_train, 10)
y_test_onehot  = tf.keras.utils.to_categorical(y_test, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [8]:
model = models.Sequential([
    layers.Dense(1024, activation='relu', input_shape=(3072,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    x_train, y_train_onehot,
    epochs=30,
    batch_size=64,
    validation_data=(x_test, y_test_onehot),
    verbose=2
)


Epoch 1/30
782/782 - 63s - 81ms/step - accuracy: 0.3169 - loss: 1.8839 - val_accuracy: 0.3787 - val_loss: 1.7287
Epoch 2/30
782/782 - 68s - 87ms/step - accuracy: 0.3993 - loss: 1.6714 - val_accuracy: 0.3960 - val_loss: 1.7219
Epoch 3/30
782/782 - 47s - 60ms/step - accuracy: 0.4318 - loss: 1.5856 - val_accuracy: 0.4460 - val_loss: 1.5571
Epoch 4/30
782/782 - 82s - 104ms/step - accuracy: 0.4587 - loss: 1.5167 - val_accuracy: 0.4599 - val_loss: 1.5225
Epoch 5/30
782/782 - 83s - 107ms/step - accuracy: 0.4716 - loss: 1.4766 - val_accuracy: 0.4806 - val_loss: 1.4663
Epoch 6/30
782/782 - 81s - 103ms/step - accuracy: 0.4836 - loss: 1.4425 - val_accuracy: 0.4659 - val_loss: 1.4920
Epoch 7/30
782/782 - 48s - 61ms/step - accuracy: 0.4962 - loss: 1.4098 - val_accuracy: 0.4864 - val_loss: 1.4586
Epoch 8/30
782/782 - 46s - 59ms/step - accuracy: 0.5073 - loss: 1.3759 - val_accuracy: 0.4900 - val_loss: 1.4487
Epoch 9/30
782/782 - 46s - 59ms/step - accuracy: 0.5188 - loss: 1.3484 - val_accuracy: 0.4949

In [10]:
# Prediksi
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = y_test.flatten()

# Evaluasi metrik
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
auc = roc_auc_score(tf.keras.utils.to_categorical(y_true, 10), y_pred_probs, average='macro', multi_class='ovr')

print("📊 Evaluasi MLP TensorFlow:")
print(f"Akurasi  : {acc:.4f}")
print(f"Presisi  : {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-Score : {f1:.4f}")
print(f"AUC      : {auc:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step
📊 Evaluasi MLP TensorFlow:
Akurasi  : 0.4847
Presisi  : 0.4835
Recall   : 0.4847
F1-Score : 0.4778
AUC      : 0.8705
